# Showcase of the "to_sql" functionality of mlinspect

It will be shown how parts of the original mlinspect example pipelines
"healthcare" and "compas" will be inspected using the additional "to_sql"
functionality.

## First install the required packages

In [1]:
# Install the pip packages in the current Jupyter kernel
import pathlib
import sys
!{sys.executable} -m pip install faker

## Some parameters you might want to set:#

In [4]:
from mlinspect.to_sql.dbms_connectors.postgresql_connector import PostgresqlConnector
from mlinspect.to_sql.dbms_connectors.umbra_connector import UmbraConnector
from pandas_connector import PandasConnector
from _benchmark_utility import REPETITIONS, plot_compare, ROOT_DIR
from _code_as_string import Join, GroupBy, Selection, Projection
import matplotlib.pyplot as plt

UMBRA_DIR = r"/home/luca/Documents/Bachelorarbeit/umbra-students"
DO_CLEANUP= True

## Operation performance comparison (not exhaustive)

In [3]:
# Based on mlinspect benchmarks.
t1_name = "histories"
t2_name = "patients"

operations = ["Join", "Select", "Project", "GroupBy"]

files = []
for i in REPETITIONS:
    path_table1 = ROOT_DIR / f"data_generation/generated_csv/healthcare_histories_generated_{i}.csv"
    path_table2 = ROOT_DIR / f"data_generation/generated_csv/healthcare_patients_generated_{i}.csv"
    files.append((path_table1, path_table2))

umbra_times = [[] for _ in operations]
postgres_times = [[] for _ in operations]
pandas_times = [[] for _ in operations]

postgres = PostgresqlConnector(dbname="healthcare_benchmark", user="luca", password="password", port=5432,
                               host="localhost")
pandas = PandasConnector()

for i, (table1, table2) in enumerate(files):
    umbra = UmbraConnector(dbname="", user="postgres", password=" ", port=5433, host="/tmp/",
                           umbra_dir=UMBRA_DIR)

    umbra.add_csv(table_name=t2_name, path_to_csv=table2, null_symbols=["?"], delimiter=",", header=True)
    umbra.add_csv(table_name=t1_name, path_to_csv=table1, null_symbols=["?"], delimiter=",", header=True)

    postgres.add_csv(table_name=t2_name, path_to_csv=table2, null_symbols=["?"], delimiter=",", header=True)
    postgres.add_csv(table_name=t1_name, path_to_csv=table1, null_symbols=["?"], delimiter=",", header=True)

    print(f"ITERATION: {i} - for table size of: {10 ** (i + 2)}")
    repetitions = 10

    input_join = t1_name, t2_name, "ssn"
    umbra_times[0].append(umbra.benchmark_run(Join.get_sql_code(*input_join), repetitions))
    postgres_times[0].append(postgres.benchmark_run(Join.get_sql_code(*input_join), repetitions))
    pandas_times[0].append(
        pandas.benchmark_run(Join.get_pandas_code(table1, table2, "ssn"), repetitions=repetitions))

    input_sel = t1_name, "complications", ">", "5"
    umbra_times[1].append(umbra.benchmark_run(Selection.get_sql_code(*input_sel), repetitions))
    postgres_times[1].append(postgres.benchmark_run(Selection.get_sql_code(*input_sel), repetitions))
    pandas_times[1].append(
        pandas.benchmark_run(Selection.get_pandas_code(table1, "complications", ">", "5"),
                             repetitions=repetitions))

    input_project = t1_name, "smoker"
    umbra_times[2].append(umbra.benchmark_run(Projection.get_sql_code(*input_project), repetitions))
    postgres_times[2].append(postgres.benchmark_run(Projection.get_sql_code(*input_project), repetitions))
    pandas_times[2].append(
        pandas.benchmark_run(Projection.get_pandas_code(table1, "smoker"), repetitions=repetitions))

    input_project = t1_name, "smoker", "complications", "AVG"
    umbra_times[3].append(umbra.benchmark_run(GroupBy.get_sql_code(*input_project), repetitions))
    postgres_times[3].append(postgres.benchmark_run(GroupBy.get_sql_code(*input_project), repetitions))
    pandas_times[3].append(
        pandas.benchmark_run(GroupBy.get_pandas_code(table1, "smoker", "complications", "mean"),
                             repetitions=repetitions))
    # in the end we have 3 lists == [[*joins*][*selections*][*projections*]]

names = ["Umbra", "Postgresql", "Pandas"]
for i, title in enumerate(operations):
    table = [umbra_times[i], postgres_times[i], pandas_times[i]]
    plot = plot_compare(title, REPETITIONS, all_y=table, all_y_names=names, save=True)

No user with password configured. For initial setup, connect via a domain socket:
   psql -h /tmp -U postgres


ITERATION: 0 - for table size of: 100
Executing Query in Umbra...
Done in 0.0002094!
Executing Query in Postgres...
Done in 0.10189999999999999!
Executing Query in Pandas...
Done in 5.210922300102538!
Executing Query in Umbra...
Done in 7.38e-05!
Executing Query in Postgres...
Done in 0.033400000000000006!
Executing Query in Pandas...
Done in 2.499655899919162!
Executing Query in Umbra...
Done in 0.00015319999999999998!
Executing Query in Postgres...
Done in 0.026500000000000003!
Executing Query in Pandas...
Done in 2.9723665000346955!
Executing Query in Umbra...
Done in 0.0003685!
Executing Query in Postgres...
Done in 0.091!
Executing Query in Pandas...
Done in 5.4432681999969645!


Killed
No user with password configured. For initial setup, connect via a domain socket:
   psql -h /tmp -U postgres


ITERATION: 1 - for table size of: 1000
Executing Query in Umbra...
Done in 0.0007430000000000001!
Executing Query in Postgres...
Done in 3.4902!
Executing Query in Pandas...
Done in 6.294666800022242!
Executing Query in Umbra...
Done in 0.0001651!
Executing Query in Postgres...
Done in 0.2374!
Executing Query in Pandas...
Done in 2.800366700103041!
Executing Query in Umbra...
Done in 0.00013700000000000002!
Executing Query in Postgres...
Done in 0.2969!
Executing Query in Pandas...
Done in 1.7781336000552983!
Executing Query in Umbra...
Done in 0.0003965!
Executing Query in Postgres...
Done in 0.6638999999999999!
Executing Query in Pandas...
Done in 4.8582624998743995!


Killed
No user with password configured. For initial setup, connect via a domain socket:
   psql -h /tmp -U postgres


ITERATION: 2 - for table size of: 10000
Executing Query in Umbra...
Done in 0.0031942999999999997!
Executing Query in Postgres...
Done in 31.891199999999998!
Executing Query in Pandas...
Done in 19.66948280005454!
Executing Query in Umbra...
Done in 0.0008033999999999999!
Executing Query in Postgres...
Done in 1.3955000000000002!
Executing Query in Pandas...
Done in 5.128223400060961!
Executing Query in Umbra...
Done in 0.0004465!
Executing Query in Postgres...
Done in 1.4773!
Executing Query in Pandas...
Done in 5.134220800027833!
Executing Query in Umbra...
Done in 0.0007681999999999999!
Executing Query in Postgres...
Done in 2.9023!
Executing Query in Pandas...
Done in 8.485655100048461!


### The original results:
**Join**:

![](./plots/Join.png)

**GroupBy**:

![](./plots/GroupBy.png)

**Select**:

![](./plots/Select.png)

**Project**:

![](./plots/Project.png)

# Data Generation

To be able to benchmark and compare the different approaches, some datasets
will need to be generated before. The datasets are just and expansion of the
original ones.

In [4]:
from data_generation._compas_data_generation import generate_compas_dataset
from data_generation._healthcare_data_generation import generate_healthcare_dataset

compas_tain, compas_test = zip(*generate_compas_dataset())
healthcare_histories, healthcare_patients = zip(*generate_healthcare_dataset())

ImportError: attempted relative import with no known parent package

In [5]:
# Clean_up:

if DO_CLEANUP:
    plot_dir = ROOT_DIR / f"plots"
    [f.unlink() for f in plot_dir.glob("*_*.png") if f.is_file()]